## PySpark and SparkSQL Basics
### How to implement Spark with Python Programming
Apache Spark is a cluster computing system that offers comprehensive libraries and APIs for developers and supports languages including Java, Python, R, and Scala. SparkSQL can be represented as the module in Apache Spark for processing unstructured data with the help of DataFrame API.
Python is revealed the Spark programming model to work with structured data by the Spark Python API which is called as PySpark.

Tutorial URL: https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53

The objective is to demonstrate how to run Spark with PySpark and execute common functions.

In [2]:
# Uncomment the script below to install libraries to be utilized in this tutorial
#import sys
#!{sys.executable} -m pip install pyarrow

In [5]:
# Importing the required packages
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

In [6]:
# Initializing a Spark Session
sc = SparkSession.builder.appName("PySparkExample")\
    .config ("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.maxResultSize","5g") \
    .config ("spark.sql.execution.arrow.enabled", "true")\
    .getOrCreate()

#### Creating Data Frames
A DataFrame can be accepted as a distributed and tabulated collection of titled columns which is similar to a table in a relational database. In this post, we will be using DataFrame operations on PySpark API while working with datasets.

The data-set for this task has been downloaded from the below mentioned Kaggle Data-set, New York Times Best Sellers.

Link: https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers

Gathered from the New York Times API for Hardcover Fiction best sellers from June 7, 2008 to July 22, 2018. The API can be found here: https://developer.nytimes.com/

Collected data includes the book title, author, the date of the best seller list, the published date of the list, the book description, the rank (this week and last week), the publisher, number of weeks on the list, and the price.

In [7]:
# Creating a spark data frame
# JSON
dataframe = sc.read.json('E:/Kaggle/PySpark/PySpark and SparkSQL Basics/nyt2.json')

In [8]:
# Viewing the data
dataframe.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|


#### Duplicate Values
Duplicate values in a table can be eliminated by using dropDuplicates() function.

In [9]:
# Checking for duplicate values and eliminating them
data_drpdup = dataframe.dropDuplicates()

In [10]:
# Data after droping duplicates
data_drpdup.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|[[1213401600000]]|Juan Cabrillo and...|[26.95,]|[[1214697600000]]|          Putnam| [4]|           [3]|         PLAGUE SHIP|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|[[1215820800000]]|Detectives Lincol...|   [, 0]|[[1217116800000]]|Simon & Schuster|[20]|           [0]|   THE BROKEN WINDOW|

In [11]:
# Viewing the number of columns
dataframe.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

#### Queries - SQL
Querying operations can be used for various purposes such as subsetting columns with “select”, adding conditions with “when” and filtering column contents with “like”.

In [12]:
# "Select" Operation
# Displaying all the authors
dataframe.select("author").show(10)

# Viewing other columns along with author
dataframe.select("author", "title", "rank", "price").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth

In [13]:
# "When" Operation
# Adding the when dependency - if else
dataframe.select("title", when(dataframe.title != 'ODD HOURS', 1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [14]:
# "isin" Operation
# Defining conditions to rows
dataframe[dataframe.author.isin("John Sandford", "Emily Griffin")].show(10)

+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+---------+----+--------------+--------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|   price|   published_date|publisher|rank|rank_last_week|         title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+---------+----+--------------+--------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|John Sandford|[[1211587200000]]|The Minneapolis d...|[26.95,]|[[1212883200000]]|   Putnam| [7]|           [4]|  PHANTOM PREY|          [3]|
|[5b4aa4ead3089013...|http://www.amazon...|John Sandford|[[1212192000000]]|The Minneapolis d...|[26.95,]|[[1213488000000]]|   Putnam| [9]|           [7]|  PHANTOM PREY|          [4]|
|[5b4aa4ead3089013...|http://www.amazon...|John Sandford|[[1212796800000]]|The Minnea

In [15]:
# "Like" Operation
# Filtering data
dataframe.select("author", "title",
                dataframe.title.like("%THE%")).show(10)

+--------------------+--------------------+----------------+
|              author|               title|title LIKE %THE%|
+--------------------+--------------------+----------------+
|       Dean R Koontz|           ODD HOURS|           false|
|     Stephenie Meyer|            THE HOST|            true|
|        Emily Giffin|LOVE THE ONE YOU'...|            true|
|   Patricia Cornwell|           THE FRONT|            true|
|     Chuck Palahniuk|               SNUFF|           false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|           false|
|       John Sandford|        PHANTOM PREY|           false|
|       Jimmy Buffett|          SWINE NOT?|           false|
|    Elizabeth George|     CARELESS IN RED|           false|
|      David Baldacci|     THE WHOLE TRUTH|            true|
+--------------------+--------------------+----------------+
only showing top 10 rows



In [16]:
# “Startswith” — “ Endswith”
dataframe.select("author", "title",
                dataframe.title.startswith("THE")).show(10)

dataframe.select("author", "title",
                dataframe.title.endswith("NT")).show(10)

+--------------------+--------------------+----------------------+
|              author|               title|startswith(title, THE)|
+--------------------+--------------------+----------------------+
|       Dean R Koontz|           ODD HOURS|                 false|
|     Stephenie Meyer|            THE HOST|                  true|
|        Emily Giffin|LOVE THE ONE YOU'...|                 false|
|   Patricia Cornwell|           THE FRONT|                  true|
|     Chuck Palahniuk|               SNUFF|                 false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|                 false|
|       John Sandford|        PHANTOM PREY|                 false|
|       Jimmy Buffett|          SWINE NOT?|                 false|
|    Elizabeth George|     CARELESS IN RED|                 false|
|      David Baldacci|     THE WHOLE TRUTH|                  true|
+--------------------+--------------------+----------------------+
only showing top 10 rows

+--------------------+--------------

In [17]:
# “Substring” Operation
dataframe.select(dataframe.author.substr(1, 3).
                 alias("title")).show(5)

dataframe.select(dataframe.author.substr(3, 6).
                 alias("title")).show(5)

dataframe.select(dataframe.author.substr(1, 6).
                 alias("title")).show(5)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
|  Pat|
|  Chu|
+-----+
only showing top 5 rows

+------+
| title|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



#### Add, Update & Remove Columns
Data manipulation functions are also available in the DataFrame API.

In [19]:
# Adding Columns
dataframe = dataframe.withColumn('new_column',
                                lit('This is a new column'))

display(dataframe)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

In [20]:
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [21]:
# Update column 'amazon_product_url' with 'URL'
dataframe = dataframe.withColumnRenamed('amazon_product_url', 'URL')
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [22]:
# Removing Columns
dataframe_remove = dataframe.drop("publisher", "published_date").show(5)

dataframe_remove2 = dataframe.drop(dataframe.publisher).drop(dataframe.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]| [2]|           [1]|            THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587

#### Inspect Data
There exist several types of functions to inspect data. Below, you can find some of the commonly used ones.

In [23]:
# Dataframe columns and data types
dataframe.dtypes

[('_id', 'struct<$oid:string>'),
 ('URL', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>'),
 ('new_column', 'string')]

In [24]:
# Display contents of dataframe
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|              author| bestsellers_date|         description|   price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|              Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens

In [25]:
# Return first n rows
dataframe.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), URL='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'), new_column='This is a new column')

In [26]:
# Return first row
dataframe.first()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), URL='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'), new_column='This is a new column')

In [27]:
# Return first n rows
dataframe.take(5)

[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), URL='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'), new_column='This is a new column'),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), URL='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_

In [28]:
# Computes summary statistics
dataframe.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+--------------------+
|summary|                 URL|         author|         description|publisher|             title|          new_column|
+-------+--------------------+---------------+--------------------+---------+------------------+--------------------+
|  count|               10195|          10195|               10195|    10195|             10195|               10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|                null|
| stddev|                null|           null|                null|     null| 370.9760613506458|                null|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|This is a new column|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|This is a new column|
+-------+--------------------+---------------+----------

In [29]:
# Returns columns of dataframe
dataframe.columns

['_id',
 'URL',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list',
 'new_column']

In [30]:
# Counts the number of rows in dataframe
dataframe.count()

10195

In [31]:
# Counts the number of distinct rows in dataframe
dataframe.distinct().count()

10195

In [35]:
# Prints plans including physical and logical
dataframe.explain()

== Physical Plan ==
*(1) Project [_id#7, amazon_product_url#8 AS URL#562, author#9, bestsellers_date#10, description#11, price#12, published_date#13, publisher#14, rank#15, rank_last_week#16, title#17, weeks_on_list#18, This is a new column AS new_column#458]
+- BatchScan[_id#7, amazon_product_url#8, author#9, bestsellers_date#10, description#11, price#12, published_date#13, publisher#14, rank#15, rank_last_week#16, title#17, weeks_on_list#18] JsonScan Location: InMemoryFileIndex[file:/E:/Kaggle/PySpark/PySpark and SparkSQL Basics/nyt2.json], ReadSchema: struct<_id:struct<$oid:string>,amazon_product_url:string,author:string,bestsellers_date:struct<$d...




#### GroupBy Operation
The grouping process is applied with GroupBy() function by adding column name in function.

In [36]:
# Group by author and count the number of books in each group
dataframe.groupBy("author").count().show(10)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Kate Jacobs|    3|
|       Karen Robards|    6|
|     Gary Shteyngart|    3|
|         Lisa Genova|    7|
|James Patterson a...|   30|
|         Ruth Reichl|    3|
|         JRR Tolkien|    2|
+--------------------+-----+
only showing top 10 rows



#### Filter Operation
Filtering is applied by using filter() function with a condition parameter added inside of it. This function is case sensitive.

In [37]:
# Filtering the entries of title
# Only keeps records having value 'THE HOST'

dataframe.filter(dataframe["title"] == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1212192000000]]|Aliens have taken...|[25.99,]|[[1213488000000]]|Little, Brown| [2]|           [2]|THE HOST|          [4]|Th

#### Missing and Replacing Values
For every dataset, there is always a need for replacing, existing values, dropping unnecessary columns and filling missing values in data preprocessing stages. pyspark.sql.DataFrameNaFunction library helps us to manipulate data with this respect. 

In [38]:
# Replacing null values
dataframe.na.fill(50).show(5)
#dataFrame.fillna()
#dataFrameNaFunctions.fill()

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [39]:
# Return new dataframe restricting rows with null values
dataframe.na.drop().show(5)
#dataFrame.dropna()
#dataFrameNaFunctions.drop()

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [40]:
# Return new dataframe replacing one value with another
dataframe.na.replace(10, 20).show(5)
#dataFrame.replace()
#dataFrameNaFunctions.replace()

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [41]:
dataframe.na.replace(['_id', 'Unique_ID'], ['amazon_product_url', 'URL'], 'name').show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

#### Repartitioning
It is possible to increase or decrease the existing level of partitioning in RDD Increasing can be actualized by using repartition(self, numPartitions) function which results in a new RDD that obtains same /higher number of partitions. Decreasing can be processed with coalesce(self, numPartitions, shuffle=False) function that results in new RDD with a reduced number of partitions to a specified number. 

In [42]:
# Dataframe with 10 partitions
dataframe.repartition(10).rdd.getNumPartitions()

10

In [43]:
# Dataframe with 1 partition
dataframe.coalesce(1).rdd.getNumPartitions()

1

#### Running SQL Queries Programmatically
Raw SQL queries can also be used by enabling the “sql” operation on our SparkSession to run SQL queries programmatically and return the result sets as DataFrame structures. 

##### Registering Data Frames

In [44]:
dataframe.createGlobalTempView("nyt2")

In [45]:
dataframe.createTempView("bestsellers")

In [46]:
dataframe.createOrReplaceTempView("nyt2")

##### Registering Tables

In [48]:
# Registering a table
dataframe.registerTempTable("df")

In [49]:
sc.sql("select * from df").show(3)


sc.sql("select \
               CASE WHEN description LIKE '%love%' THEN 'Love_Theme' \
               WHEN description LIKE '%hate%' THEN 'Hate_Theme' \
               WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \
               WHEN description LIKE '%anger%' THEN 'Anger_Theme' \
               WHEN description LIKE '%horror%' THEN 'Horror_Theme' \
               WHEN description LIKE '%death%' THEN 'Criminal_Theme' \
               WHEN description LIKE '%detective%' THEN 'Mystery_Theme' \
               ELSE 'Other_Themes' \
               END Themes \
       from df").groupBy('Themes').count().show()

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Bro

#### Output

##### Data Structures
DataFrame API uses RDD as a base and it converts SQL queries into low-level RDD functions. By using .rdd operation, a dataframe can be converted into RDD. It is also possible to convert Spark Dataframe into a string of RDD and Pandas formats.

In [50]:
# Converting dataframe into an RDD
rdd_convert = dataframe.rdd

In [51]:
# Converting dataframe into a RDD of string 
dataframe.toJSON().first()

'{"_id":{"$oid":"5b4aa4ead3089013507db18b"},"URL":"http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20","author":"Dean R Koontz","bestsellers_date":{"$date":{"$numberLong":"1211587200000"}},"description":"Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.","price":{"$numberInt":"27"},"published_date":{"$date":{"$numberLong":"1212883200000"}},"publisher":"Bantam","rank":{"$numberInt":"1"},"rank_last_week":{"$numberInt":"0"},"title":"ODD HOURS","weeks_on_list":{"$numberInt":"1"},"new_column":"This is a new column"}'

In [52]:
# Obtaining contents of df as Pandas 
dataframe.toPandas()

C:\Users\alina\Anaconda3\envs\myenv\lib\site-packages\pyspark\sql\pandas\conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,_id,URL,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list,new_column
0,"(5b4aa4ead3089013507db18b,)",http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"((1211587200000,),)","Odd Thomas, who can communicate with the dead,...","(None, 27)","((1212883200000,),)",Bantam,"(1,)","(0,)",ODD HOURS,"(1,)",This is a new column
1,"(5b4aa4ead3089013507db18c,)",http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,"((1211587200000,),)",Aliens have taken control of the minds and bod...,"(25.99, None)","((1212883200000,),)","Little, Brown","(2,)","(1,)",THE HOST,"(3,)",This is a new column
2,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)",This is a new column
3,"(5b4aa4ead3089013507db18e,)",http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,"((1211587200000,),)",A Massachusetts state investigator and his tea...,"(22.95, None)","((1212883200000,),)",Putnam,"(4,)","(0,)",THE FRONT,"(1,)",This is a new column
4,"(5b4aa4ead3089013507db18f,)",http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,"((1211587200000,),)",An aging porn queens aims to cap her career by...,"(24.95, None)","((1212883200000,),)",Doubleday,"(5,)","(0,)",SNUFF,"(1,)",This is a new column
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10190,"(5b4aa4ead3089013507dd959,)",https://www.amazon.com/Clancy-Line-Sight-Jack-...,Mike Maden,"((1530921600000,),)",Jack Ryan Jr. risks his life to protect a woma...,"(None, 0)","((1532217600000,),)",Putnam,"(11,)","(6,)",TOM CLANCY LINE OF SIGHT,"(4,)",This is a new column
10191,"(5b4aa4ead3089013507dd95a,)",https://www.amazon.com/Something-Water-Novel-C...,Catherine Steadman,"((1530921600000,),)",A documentary filmmaker and an investment bank...,"(None, 0)","((1532217600000,),)",Ballantine,"(12,)","(11,)",SOMETHING IN THE WATER,"(5,)",This is a new column
10192,"(5b4aa4ead3089013507dd95b,)",https://www.amazon.com/Little-Fires-Everywhere...,Celeste Ng,"((1530921600000,),)",An artist upends a quiet town outside Cleveland.,"(None, 0)","((1532217600000,),)",Penguin Press,"(13,)","(12,)",LITTLE FIRES EVERYWHERE,"(41,)",This is a new column
10193,"(5b4aa4ead3089013507dd95c,)",https://www.amazon.com/Shelter-Place-Nora-Robe...,Nora Roberts,"((1530921600000,),)",Survivors of a mass shooting outside a mall in...,"(None, 0)","((1532217600000,),)",St. Martin's,"(14,)","(5,)",SHELTER IN PLACE,"(6,)",This is a new column


##### Write & Save to Files
Any data source type that is loaded to our code as data frames can easily be converted and saved into other types including .parquet and .json.

In [53]:
# Write & Save File in .parquet format
dataframe.select("author", "title", "rank", "description")\
                .write \
                .save("E:/Kaggle/PySpark/PySpark and SparkSQL Basics/Rankings_Descriptions.parquet")

In [54]:
# Write & Save File in .json format
dataframe.select("author", "title") \
                .write \
                .save("E:/Kaggle/PySpark/PySpark and SparkSQL Basics/Authors_Titles.json",format="json")

##### Stop Spark Session

In [55]:
# End Spark Session
sc.stop()